<a href="https://colab.research.google.com/github/Ibrah-N/BMHS-School-Automation-Project/blob/main/Large_Equipment_Detection_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pdf2Image

In [1]:
!pip install -q PyMuPDF Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 52.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


# Raw Dataset Creation

In [ ]:
!unzip /content/drive/MyDrive/Symbol_Detection/Equipment_Detection/Equipment\ patches.zip

In [ ]:
# -- data paths --

data_01 = "/content/Equipment patches/Batch 1"
data_02 = "/content/Equipment patches/Batch 2 - Kalyani"
data_03 = "/content/Equipment patches/Batch 3 - Colored - Dinesh"
data_05 = "/content/Equipment patches/Batch 5 - Petro"
data_06 = "/content/Equipment patches/Batch 6 - Pack A and B"
data_07 = "/content/Equipment patches/Batch 7 - Ok"
data_08 = "/content/Equipment patches/Batch 8"

In [ ]:
# deleteing folder samle script

!rm -rf /content/bw_images/
!rm -rf /content/__MACOSX

In [ ]:
import fitz               # PyMuPDF
from PIL import Image, ImageFilter
import os
import shutil

def pdf_to_jpg(pdf_path, output_folder="output_images", dpi=300,
               threshold=220, output_folder_bw="bw_imgs", pdf_id=0, dir_id=0, blur_radius=0.5):
    """
    Convert each page of a PDF to:
      • color_image_<page>.jpg
      • bw_image_<page>.jpg

    Args:
      pdf_path (str): path to .pdf file
      output_folder (str): where to save JPEGs
      dpi (int): rendering resolution
      threshold (0–255): gray cutoff for B/W
      blur_radius (float): Gaussian blur radius on B/W
    """
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(output_folder_bw, exist_ok=True)
    doc = fitz.open(pdf_path)
    page_count = doc.page_count

    for i, page in enumerate(doc, start=1):
        pix = page.get_pixmap(dpi=dpi)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # — Color JPEG
        color_file = os.path.join(output_folder, f"color_image_{i}.jpg")
        img.save(color_file, "JPEG", quality=100, dpi=(dpi, dpi))

        # — Grayscale → threshold → blur → B/W JPEG
        gray = img.convert("L")
        bw = gray.point(lambda x: 255 if x > threshold else 0)
        bw = bw.filter(ImageFilter.GaussianBlur(blur_radius))
        bw_file = os.path.join(output_folder_bw, f"bw_image_{dir_id}_{pdf_id}_{i}.jpg")
        bw.save(bw_file, "JPEG", quality=100, dpi=(dpi, dpi))

    doc.close()
    print(f"Done! For pdf {pdf_id} Saved {page_count} page(s) to '{output_folder}'")

if __name__ == "__main__":

  # -- single pdf parse --
  # pdf_path = "/content/90-QP20-R-001_1_Process & Inst_Diagram _PID_Legend_and_Sybmols.pdf"
  # pdf_to_jpg(pdf_path,
  #             output_folder="/content/output_images")


  # -- iterate over pdf/img files --
  input_dir = "/content/xyz"
  output_bw = "/content/bw_images/"
  os.makedirs(output_bw, exist_ok=True)


  # os.listdir(input_dir)
  # -- iterate over dir --
  for dirIdx, dir in enumerate(os.listdir(input_dir)):
    if dir.endswith("DS_Store"): continue
    dir_path = os.path.join(input_dir, dir)
    print("Processing direcotry: {}".format(dir_path))
    os.makedirs(os.path.join(output_bw, dir), exist_ok=True)
    output_path = os.path.join(output_bw, dir)

    # -- iterate over sub dir --
    for idx, file_x in enumerate(os.listdir(dir_path)):

      # -- check if pdf --
      if (file_x.endswith(".pdf") or file_x.endswith(".PDF")):
        pdf_path = os.path.join(dir_path, file_x)
        pdf_to_jpg(pdf_path,
                  output_folder=output_path,
                  pdf_id = idx,
                  dir_id = dirIdx,
                  output_folder_bw=output_path)

      # -- check if image --
      elif (file_x.endswith(".jpg") or file_x.endswith(".JPG")):
        src_path = os.path.join(dir_path, file_x)
        dst_path = os.path.join(output_path, file_x)
        print(src_path, dst_path)
        shutil.copy(src_path, dst_path)
        print(f"Done! For :{file_x} Copied to {dst_path}")

      # -- else skip --
      else:
        print("Not a pdf or jpeg file -- file {}".format(file_x))


In [ ]:
import cv2
import numpy as np


# -- configuration --
input_path = "/content/bw_images"
output_path = "/content/Epuipment/"
os.makedirs(output_path, exist_ok=True)

# -- iterate over dir --
for dirIdx, dir in enumerate(os.listdir(input_path)):
  if dir.endswith("DS_Store"): continue
  dir_path = os.path.join(input_path, dir)
  output_dir_path = os.path.join(output_path, dir)
  os.makedirs(output_dir_path, exist_ok=True)
  print("Processing direcotry: {}".format(dir_path))

  for img_idx, file in enumerate(os.listdir(dir_path)):
    img_path = os.path.join(dir_path, file)
    read_image = cv2.imread(img_path)
    resized_image = cv2.resize(read_image, (3584, 3584))
    output_save = os.path.join(output_dir_path, file)
    cv2.imwrite(output_save, resized_image)

Processing direcotry: /content/bw_images/Batch 1
Processing direcotry: /content/bw_images/Batch 6 - Pack A and B
Processing direcotry: /content/bw_images/Batch 5 - Petro
Processing direcotry: /content/bw_images/Batch 2 - Kalyani
Processing direcotry: /content/bw_images/Batch 7 - Ok
Processing direcotry: /content/bw_images/Batch 8
Processing direcotry: /content/bw_images/Batch 3 - Colored - Dinesh


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!zip -r /content/equipment.zip /content/Epuipment/

In [ ]:
!cp -r /content/equipments.zip -d /content/drive/MyDrive/Symbol_Detection/Equipment_Detection/raw_datasets

cp: cannot stat '/content/equipments.zip': No such file or directory


# Model Training

## Helpers

In [ ]:
!unzip -q /content/drive/MyDrive/Symbol_Detection/Equipment_Detection/Correct_Annotated_Dataset/Equipment_V2.v1i.yolov8.zip -d /content/dataset

In [4]:
#############################################################
#    D O W N L O A D - E Q U I P M E N T - D A T A S E T    #
#############################################################
# download equipment dataset from roboflow
!pip install -q roboflow

# -- imports --
from google.colab import userdata
from roboflow import Roboflow


# -- download dataset --
rf = Roboflow(api_key=str(userdata.get('r_b')))
project = rf.workspace("pnid-relgx").project("equipment_v2-y1q1h")
version = project.version(5)
dataset = version.download("yolov8")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.8 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Equipment_V2-5 in yolov8:: 100%|██████████| 1759/1759 [00:07<00:00, 229.78it/s]


In [ ]:
#############################################################
#                                                           #
#    V E R I F Y - D A T A S E T - A N N O T A T I O N S    #
#                                                           #
#############################################################
# script for verifying the dataset annotation after labeling
# format of input data (yolo)
#   - images (.jpg, .png ....)
#   - labels (yolo_format_annotations(cls, x_cen, y_cen, width, height)) # normalized values



# == IMPORTS ==
import os
import cv2
import yaml


# ==== CONFIG ====
images_path = "/content/Equipment_Dataset/train/images"
labels_path = "/content/Equipment_Dataset/train/labels"
yaml_path = "/content/Equipment_Dataset/data.yaml"
write_img_dir = "/content/output_images"


# ==== Fixed list of 80 bright visible colors ====
colors = [
    (0, 0, 255), (255, 0, 0), (0, 255, 0), (255, 255, 0), (0, 255, 255),
    (255, 0, 255), (255, 128, 0), (128, 0, 255), (0, 128, 255), (128, 255, 0),
    (255, 0, 128), (0, 255, 128), (128, 255, 255), (255, 128, 128), (128, 128, 255),
    (255, 255, 128), (128, 255, 128), (255, 128, 255), (192, 0, 0), (0, 192, 0),
    (0, 0, 192), (192, 192, 0), (0, 192, 192), (192, 0, 192), (255, 64, 0),
    (64, 0, 255), (0, 64, 255), (64, 255, 0), (255, 0, 64), (0, 255, 64),
    (64, 255, 255), (255, 200, 0), (200, 0, 255), (0, 200, 255), (200, 255, 0),
    (255, 0, 200), (0, 255, 200), (200, 255, 255), (255, 100, 50), (50, 100, 255),
    (100, 50, 255), (50, 255, 100), (255, 50, 100), (100, 255, 50), (255, 150, 0),
    (150, 0, 255), (0, 150, 255), (150, 255, 0), (255, 0, 150), (0, 255, 150),
    (150, 255, 255), (255, 180, 75), (75, 180, 255), (180, 75, 255), (75, 255, 180),
    (255, 75, 180), (180, 255, 75), (230, 30, 30), (30, 230, 30), (30, 30, 230),
    (230, 230, 30), (30, 230, 230), (230, 30, 230), (255, 210, 60), (60, 210, 255),
    (210, 60, 255), (60, 255, 210), (255, 60, 210), (210, 255, 60), (255, 80, 160),
    (80, 160, 255), (160, 80, 255), (80, 255, 160), (255, 160, 80), (160, 255, 80),
    (100, 255, 200), (200, 100, 255), (255, 200, 100), (100, 200, 255), (200, 255, 100)
]

# ==== Ensure output directory exists ====
os.makedirs(write_img_dir, exist_ok=True)


# ==== Read Yaml files ====
classes = []
with open(yaml_path, "r") as f:
    yaml_data = yaml.safe_load(f)
    classes = yaml_data.get("names", [])


# ==== Parse images + draw labels ====
for f_idx, img_file in enumerate(os.listdir(images_path)):
    if not img_file.lower().endswith(('.jpg', '.png', '.jpeg')):
        continue

    image_path = os.path.join(images_path, img_file)
    label_path = os.path.join(labels_path, os.path.splitext(img_file)[0] + ".txt")

    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image: {image_path}")
        continue

    h, w = img.shape[:2]
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()

        for line in lines:
            parts = line.strip().split()
            if len(parts) != 5:
                continue

            class_id, x_center, y_center, bbox_width, bbox_height = map(float, parts)
            class_id = int(class_id)

            if len(classes) > 0:
              class_name = classes[class_id] if 0 <= class_id < len(classes) else "Unknown"
            else:
              class_name = str(class_id)

            # === Denormalize YOLO coords ===
            x_center *= w
            y_center *= h
            bbox_width *= w
            bbox_height *= h

            x1 = int(x_center - bbox_width / 2)
            y1 = int(y_center - bbox_height / 2)
            x2 = int(x_center + bbox_width / 2)
            y2 = int(y_center + bbox_height / 2)

            # === Pick color (wrap around if > 80 classes) ===
            color = colors[class_id % len(colors)]


            # === Draw rectangle + class ID ===
            cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.putText(img, f"H: {bbox_height} - W: {bbox_width} - Tag: {class_name}", (x1, max(20, y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    if f_idx == 20:
      break


    # save verified image
    output_path = os.path.join(write_img_dir, img_file)
    cv2.imwrite(output_path, img)
    print(f"File No: {f_idx} - Saved to {output_path}")

In [ ]:
!rm -rf /content/output_images

In [ ]:
_# remove the visualization directory
!rm -rf /content/yolov9/Equipment_Dataset


## Prepare Dataset


- Yolo Format Dataset
  - dataset
      - train
        - images
        - labels
      - val
        - images
        - labels



- The Image and labels file name should be same
- The images to be store within the images directory (img.jpg)
- labels format (img.txt)
  - each labels should have the following structure
  - class, xc, yc, w, h      <---  Normalized values

In [ ]:
%cd /content

/content


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
######################################################
#  C R E A T E - D E T E C T I O N - D A T A S E T   #
######################################################
# from all above preprocessing the data now creating a
# detection dataset which is supported by the detection
# models like yolo, yolox, yolonas, gelan-e, gelan-c etc


# -- dataset structure --
root_dir = "Equipment_Dataset"
root_dir_train_images = os.path.join(root_dir, "train", "images")
root_dir_train_labels = os.path.join(root_dir, "train", "labels")
root_dir_valid_images = os.path.join(root_dir, "valid", "images")
root_dir_valid_labels = os.path.join(root_dir, "valid", "labels")


# -- create datasets dirs --
os.makedirs(root_dir, exist_ok=True)
os.makedirs(root_dir_train_images, exist_ok=True)
os.makedirs(root_dir_train_labels, exist_ok=True)
os.makedirs(root_dir_valid_images, exist_ok=True)
os.makedirs(root_dir_valid_labels, exist_ok=True)

In [ ]:
################################################################
#  C O P Y - P R O C E S S E D - D A T A - TO - D A T A S E T  #
################################################################
# copy the processed data (patched images and labels) to the
# actual detection dataset, which then to be used for training


import numpy as np
import glob
import os
import shutil


# -- dataset paths --
datasets = [
    ("/content/dataset/train/images/", "/content/dataset/train/labels/")
]



# -- move data to train directory --
for img_dir, label_dir in datasets:
    for img_file in glob.glob(f"{img_dir}/*"):
        shutil.move(img_file, root_dir_train_images)
    for label_file in glob.glob(f"{label_dir}/*"):
        shutil.move(label_file, root_dir_train_labels)



print("Images Samples : {}  -   Labels Samples : {}".format(
    len(os.listdir(root_dir_train_images)),
    len(os.listdir(root_dir_train_labels))))

Images Samples : 1186  -   Labels Samples : 1186


In [ ]:
#################################################################
# C R E A T E - T R A I N -  & - V A L I D A T I O N - S E T S  #
#################################################################
# after copying all images into train_images of detection dataset
# now have to copy some random samples into validation dataset
import random

# -- list files --
raw_images = os.listdir(root_dir_train_images)
raw_labels = os.listdir(root_dir_train_labels)
raw_images.sort()
raw_labels.sort()



# -- Separate Samples For Validation --
np.random.seed(42)
num_samples = int(50) # if percentage used... 4 per%
selected_indices = np.random.choice(len(raw_images), size=num_samples, replace=False)
random_raw_valid_images = [raw_images[i] for i in selected_indices]
random_raw_valid_labels = [raw_labels[i] for i in selected_indices]



# -- Move data to Validation directories --
for i in range(len(random_raw_valid_images)):
    src_img_path = os.path.join(root_dir_train_images, random_raw_valid_images[i])
    dst_img_path = os.path.join(root_dir_valid_images, random_raw_valid_images[i])
    src_lbl_path = os.path.join(root_dir_train_labels, random_raw_valid_labels[i])
    dst_lbl_path = os.path.join(root_dir_valid_labels, random_raw_valid_labels[i])

    shutil.move(src_img_path, dst_img_path)
    shutil.move(src_lbl_path, dst_lbl_path)


print("Training Samples: Images: {} - Labels {}".format(
    len(os.listdir(root_dir_train_images)),
    len(os.listdir(root_dir_train_labels))))
print("Validation Samples: Images: {} - Labels {}".format(
    len(os.listdir(root_dir_valid_images)),
    len(os.listdir(root_dir_valid_labels))))

Training Samples: Images: 1136 - Labels 1136
Validation Samples: Images: 50 - Labels 50


### Augment: 90 Degree

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A

# Input & Output Directories
image_dir = "/content/Equipment_Dataset/train/images"  # Folder with original images
label_dir = "/content/Equipment_Dataset/train/labels"  # Folder with YOLO labels
output_image_dir = "/content/Equipment_Dataset/train/images"  # Save rotated images
output_label_dir = "/content/Equipment_Dataset/train/labels"  # Save updated labels

# Ensure output directories exist
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)


images_files_list = np.random.choice(os.listdir(image_dir), size=500, replace=False)

# Define augmentation (90-degree rotation)
transform = A.Compose([
    A.Rotate(limit=[90, 90], p=1)  # Rotate exactly 90 degrees
])

def correct_yolo_labels_after_rotation(label_path, img_width, img_height):
  """ Adjust YOLO bounding boxes after a 90-degree clockwise rotation. """
  new_labels = []
  with open(label_path, "r") as file:
    lines = file.readlines()
  for line in lines:
    data = line.strip().split()
    class_id, x_center, y_center, bbox_w, bbox_h = map(float, data)

    # Convert to absolute values
    x_center_abs = x_center * img_width
    y_center_abs = y_center * img_height
    bbox_w_abs = bbox_w * img_width
    bbox_h_abs = bbox_h * img_height

    # Transform coordinates for 90-degree rotation
    new_x_center_abs = y_center_abs
    new_y_center_abs = img_width - x_center_abs
    new_bbox_w_abs, new_bbox_h_abs = bbox_h_abs, bbox_w_abs  # Swap width & height

    # Normalize back to YOLO format
    new_x_center = new_x_center_abs / img_height
    new_y_center = new_y_center_abs / img_width
    new_bbox_w = new_bbox_w_abs / img_height
    new_bbox_h = new_bbox_h_abs / img_width

    new_labels.append(f"{int(class_id)} {new_x_center} {new_y_center} {new_bbox_w} {new_bbox_h}\n")
  return new_labels



# Process all images
for img_idx, img_file in enumerate(images_files_list):
  if img_file.endswith(".jpg") or img_file.endswith(".png"):
    img_path = os.path.join(image_dir, img_file)
    label_path = os.path.join(label_dir, img_file.replace(".jpg", ".txt").replace(".png", ".txt"))

    # Load Image
    image = cv2.imread(img_path)
    img_height, img_width = image.shape[:2]
    # Rotate Image
    augmented = transform(image=image)
    rotated_image = augmented["image"]
    # Save rotated image
    rotated_img_path = os.path.join(output_image_dir, f"aug_0_{img_file}")
    cv2.imwrite(rotated_img_path, rotated_image)
    # Adjust & Save YOLO labels
    if os.path.exists(label_path):
      try:
        new_labels = correct_yolo_labels_after_rotation(label_path, img_width, img_height)
      except Exception as e:
        print(f"Error processing {img_file}: {e}")
        continue
      rotated_label_path = os.path.join(output_label_dir, f"aug_0_{img_file.replace('.jpg', '.txt').replace('.png', '.txt')}")

      with open(rotated_label_path, "w") as file:
          file.writelines(new_labels)
    print(f"Processed - {img_idx}: {img_file}")
print("Augmentation completed for all images.")

Processed - 0: bw_image_7_225_1_jpg.rf.5bddc010d2803fb626d8e6a80b9a7dfc.jpg
Processed - 1: bw_image_7_205_1_jpg.rf.dc04f47e746db075703afdf45413202b.jpg
Processed - 2: bw_image_4_123_1_jpg.rf.d23ca3d9a5948d40f16db27d7f6402db.jpg
Processed - 3: bw_image_2_76_1_jpg.rf.3c80c0bac134d79eda3b2a17c66a55d9.jpg
Processed - 4: patch_1_30_jpg.rf.72750675fd26d05fb20eaa413f7d14fe.jpg
Processed - 5: patch_2_8_jpg.rf.4e1a340e87af7a0eae47339aedf85167.jpg
Processed - 6: bw_image_4_5_13_jpg.rf.e0dce13e6d39dd8cbef6b940ffc032b9.jpg
Processed - 7: bw_image_4_147_1_jpg.rf.c6ca400c4bb54b4b7b6fc701dcfc165b.jpg
Processed - 8: bw_image_5_208_1_jpg.rf.8396239062c7a087f5a7d01d0c6dd6d2.jpg
Processed - 9: patch_1_1061_jpg.rf.788e243c6ee51b8425c8f8e042f0e1a3.jpg
Processed - 10: bw_image_7_86_1_jpg.rf.3ef806079a918c41676f3a92cebf8299.jpg
Processed - 11: patch_0_715_jpg.rf.bf9c886ee3f6a3cbc670266ef7d8e2f9.jpg
Processed - 12: patch_2_9_jpg.rf.e760da5a0df0265312f1f4acdc24e013.jpg
Processed - 13: bw_image_4_36_1_jpg.rf.77

In [ ]:
#################################################################
#   S E T - C L A S S E S - O F - Equipment - T O - Equipment   #
#################################################################
# for testing purpose to set all lines differnt classes into one
# single class of "Line" - will be better for testing detection

# -- paths configurations --
labels_train = "/content/Equipment_Dataset/train/labels"
labels_valid = "/content/Equipment_Dataset/valid/labels"



# -- iterate train labels --
for idx, file in enumerate(os.listdir(labels_train)):

  # -- paths configs --
  read_path = os.path.join(labels_train, file)
  write_path = os.path.join(labels_train, file)

  # -- read labels --
  with open(read_path, "r") as f:
    lines = f.readlines()

  # -- write labels --
  with open(write_path, "w") as f:
    for line in lines:
      parts = line.strip().split()
      if len(parts) == 5:
        parts[0] = "0"
        f.write(" ".join(parts) + "\n")
print("For Training Set - Labels Class Successfully Set to '0: Equipment'")




# -- iterate valid labels --
for idx, file in enumerate(os.listdir(labels_valid)):

  # -- paths configs --
  read_path = os.path.join(labels_valid, file)
  write_path = os.path.join(labels_valid, file)

  # -- read labels --
  with open(read_path, "r") as f:
    lines = f.readlines()

  # -- write labels --
  with open(write_path, "w") as f:
    for line in lines:
      parts = line.strip().split()
      if len(parts) == 5:
        parts[0] = "0"
        f.write(" ".join(parts) + "\n")


print("For Valid Set - Labels Class Successfully Set to Claass 'Equipment: 0'")

For Training Set - Labels Class Successfully Set to '0: Equipment'
For Valid Set - Labels Class Successfully Set to Claass 'Equipment: 0'


In [ ]:
#################################################################
#    C R E A T E - M A N U A L - D A T A . Y A M L - F I L E    #
#################################################################
# create manuall data.yaml file for the training of detector
import yaml

data = {
    'names': ['Equipment'],
    'nc': 1,
    'train': './Equipment_Dataset/train/images',
    'val': './Equipment_Dataset/valid/images'
}

with open('/content/Equipment_Dataset/data.yaml', 'w') as file:
    yaml.dump(data, file, sort_keys=False)

In [ ]:
import os

# -- dataset dirs structure --
root_dataset_dir = "/content/Equipment_Dataset"
root_train_dir = "/content/Equipment_Dataset/train"
root_val_dir = "/content/Equipment_Dataset/valid"



print("Training Images & Labels")
print(len(os.listdir(f"{root_train_dir}/images")))
print(len(os.listdir(f"{root_train_dir}/labels")))


print("\nValidation Images & Labels")
print(len(os.listdir(f"{root_val_dir}/images")))
print(len(os.listdir(f"{root_val_dir}/labels")))

Training Images & Labels
1136
1136

Validation Images & Labels
50
50


## Model Training

In [ ]:
# clone the github repo for training

!git clone https://github.com/SkalskiP/yolov9.git
%cd yolov9

!pip install -r requirements.txt -q
%cd /content/

Cloning into 'yolov9'...
remote: Enumerating objects: 325, done.
remote: Total 325 (delta 0), reused 0 (delta 0), pack-reused 325 (from 1)
Receiving objects: 100% (325/325), 2.25 MiB | 45.10 MiB/s, done.
Resolving deltas: 100% (162/162), done.
/content/yolov9
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 65.0 MB/s eta 0:00:00
/content


In [ ]:
# copy the weights if already trained
# or fine-tuned weights availabe in drive

!mkdir /content/weights/
!cp -r /content/drive/MyDrive/Lines_Detection/models/* /content/weights/
!ls -la /content/weights/

In [ ]:
# download pretrained weights (imagenet trained)

!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-c.pt
!wget -P weights -q https://github.com/WongKinYiu/yolov9/releases/download/v0.1/gelan-e.pt

In [ ]:
# copy dataset into yolov9 repo
!cp -r /content/Equipment_Dataset/ /content/yolov9

In [ ]:
# start training
%cd /content/yolov9

# input_shape: 3584, 3584, 3
!python train.py \
--batch 1 --epochs 65 --img 3584 --min-items 0 --workers 4 --close-mosaic 13 \
--data /content/yolov9/Equipment_Dataset/data.yaml \
--weights /content/weights/gelan-c.pt \
--cfg models/detect/gelan-c.yaml \
--hyp hyp.scratch-high.yaml \
--noplots

/content/yolov9
2025-09-09 06:06:57.502713: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-09 06:06:57.521386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757398017.543129   10163 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757398017.549697   10163 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757398017.566701   10163 computation_placer.cc:177] computation placer already registered. Please check linkage an

In [ ]:
!cp /content/yolov9/runs/train/exp/weights/best.pt /content/drive/MyDrive/Lines_Detection/Correct_Annotated_Data

## Model Testing

In [ ]:
import fitz               # PyMuPDF
from PIL import Image, ImageFilter
import os
import shutil

def pdf_to_jpg(pdf_path, output_folder="output_images", dpi=300,
               threshold=220, output_folder_bw="bw_imgs", pdf_id=0, dir_id=0, blur_radius=0.5):
    """
    Convert each page of a PDF to:
      • color_image_<page>.jpg
      • bw_image_<page>.jpg

    Args:
      pdf_path (str): path to .pdf file
      output_folder (str): where to save JPEGs
      dpi (int): rendering resolution
      threshold (0–255): gray cutoff for B/W
      blur_radius (float): Gaussian blur radius on B/W
    """
    os.makedirs(output_folder, exist_ok=True)
    os.makedirs(output_folder_bw, exist_ok=True)
    doc = fitz.open(pdf_path)
    page_count = doc.page_count

    for i, page in enumerate(doc, start=1):
        pix = page.get_pixmap(dpi=dpi)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

        # — Color JPEG
        color_file = os.path.join(output_folder, f"color_image_{i}.jpg")
        img.save(color_file, "JPEG", quality=100, dpi=(dpi, dpi))

        # — Grayscale → threshold → blur → B/W JPEG
        gray = img.convert("L")
        bw = gray.point(lambda x: 255 if x > threshold else 0)
        bw = bw.filter(ImageFilter.GaussianBlur(blur_radius))
        bw_file = os.path.join(output_folder_bw, f"bw_image_{dir_id}_{pdf_id}_{i}.jpg")
        bw.save(bw_file, "JPEG", quality=100, dpi=(dpi, dpi))

    doc.close()
    print(f"Done! For pdf {pdf_id} Saved {page_count} page(s) to '{output_folder}'")

if __name__ == "__main__":

  # -- single pdf parse --
  # pdf_path = "/content/90-QP20-R-001_1_Process & Inst_Diagram _PID_Legend_and_Sybmols.pdf"
  # pdf_to_jpg(pdf_path,
  #             output_folder="/content/output_images")


  # -- iterate over pdf/img files --
  input_dir = "/content/Equipment patches"
  output_bw = "/content/bw_images/"
  os.makedirs(output_bw, exist_ok=True)


  # os.listdir(input_dir)
  # -- iterate over dir --
  for dirIdx, dir in enumerate(os.listdir(input_dir)):
    if dir.endswith("DS_Store"): continue
    dir_path = os.path.join(input_dir, dir)
    print("Processing direcotry: {}".format(dir_path))
    os.makedirs(os.path.join(output_bw, dir), exist_ok=True)
    output_path = os.path.join(output_bw, dir)

    # -- iterate over sub dir --
    for idx, file_x in enumerate(os.listdir(dir_path)):

      # -- check if pdf --
      if (file_x.endswith(".pdf") or file_x.endswith(".PDF")):
        pdf_path = os.path.join(dir_path, file_x)
        pdf_to_jpg(pdf_path,
                  output_folder=output_path,
                  pdf_id = idx,
                  dir_id = dirIdx,
                  output_folder_bw=output_path)

      # -- check if image --
      elif (file_x.endswith(".jpg") or file_x.endswith(".JPG")):
        src_path = os.path.join(dir_path, file_x)
        dst_path = os.path.join(output_path, file_x)
        print(src_path, dst_path)
        shutil.copy(src_path, dst_path)
        print(f"Done! For :{file_x} Copied to {dst_path}")

      # -- else skip --
      else:
        print("Not a pdf or jpeg file -- file {}".format(file_x))


In [ ]:
import cv2
import numpy as np


# -- configuration --
input_path = "/content/bw_images"
output_path = "/content/Epuipment/"
os.makedirs(output_path, exist_ok=True)

# -- iterate over dir --
for dirIdx, dir in enumerate(os.listdir(input_path)):
  if dir.endswith("DS_Store"): continue
  dir_path = os.path.join(input_path, dir)
  output_dir_path = os.path.join(output_path, dir)
  os.makedirs(output_dir_path, exist_ok=True)
  print("Processing direcotry: {}".format(dir_path))

  for img_idx, file in enumerate(os.listdir(dir_path)):
    img_path = os.path.join(dir_path, file)
    read_image = cv2.imread(img_path)
    resized_image = cv2.resize(read_image, (3584, 3584))
    output_save = os.path.join(output_dir_path, file)
    cv2.imwrite(output_save, resized_image)

In [ ]:
%cd /content/yolov9
!python detect.py \
--img 2560 --conf 0.1 \
--weights /content/best-3.pt \
--source /content/Equipment_V2-4/valid/images

/content/yolov9
detect: weights=['/content/best-3.pt'], source=/content/Equipment_V2-4/valid/images, data=data/coco128.yaml, imgsz=[2560, 2560], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 1e33dbb Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
gelan-e summary: 930 layers, 58002835 parameters, 0 gradients, 190.8 GFLOPs
image 1/14 /content/Equipment_V2-4/valid/images/bw_image_2_25_1_jpg.rf.392cecbab4c762046e45563b5f65e782.jpg: 2560x2560 5 Equipments, 961.0ms
image 2/14 /content/Equipment_V2-4/valid/images/bw_image_2_64_1_jpg.rf.10d51ad61f616eef862501f83d450c4f.jpg: 2560x2560 5 Equipments, 956.3ms
image 3/14 /content/Equipment_V2-4/valid/images/

In [ ]:
!zip -r equipment_results.zip /content/yolov9/runs/detect/exp3

  adding: content/yolov9/runs/detect/exp3/ (stored 0%)
  adding: content/yolov9/runs/detect/exp3/bw_image_2_73_1_jpg.rf.51e391552103ce5f9901316c45a98afe.jpg (deflated 17%)
  adding: content/yolov9/runs/detect/exp3/bw_image_7_293_1_jpg.rf.9a5241ae6617b6b4f5d97ac6398e8563.jpg (deflated 17%)
  adding: content/yolov9/runs/detect/exp3/patch_2_12_jpg.rf.4ccba53baf083f492f4e6607e227a341.jpg (deflated 9%)
  adding: content/yolov9/runs/detect/exp3/bw_image_5_42_1_jpg.rf.cb19fbaa603447791e32ce374f5fa9af.jpg (deflated 16%)
  adding: content/yolov9/runs/detect/exp3/bw_image_7_273_1_jpg.rf.6d59d1a5ba9304d4a7c6862053de424c.jpg (deflated 14%)
  adding: content/yolov9/runs/detect/exp3/bw_image_5_104_1_jpg.rf.93c0f68ebca253245c7f93a8932700cf.jpg (deflated 15%)
  adding: content/yolov9/runs/detect/exp3/patch_1_1000_jpg.rf.13c4561a52538e3f0d8c9b47130523fe.jpg (deflated 31%)
  adding: content/yolov9/runs/detect/exp3/bw_image_7_12_1_jpg.rf.0dedf78fdebdb92ba7865255e2a9c42b.jpg (deflated 10%)
  adding: conte

In [ ]:
!cp -r /content/yolov9/lines_test_01.zip /content/drive/MyDrive/Lines_Detection/Correct_Annotated_Data

## Model Parallalization Experiments